In [10]:
import requests
import os
from dotenv import load_dotenv
from authlib.integrations.requests_client import OAuth2Session

# Fetch credentials from environment
load_dotenv()
BLIZZARD_API_CLIENT_ID = os.getenv("BLIZZARD_API_CLIENT_ID")
BLIZZARD_API_CLIENT_SECRET = os.getenv("BLIZZARD_API_CLIENT_SECRET")
BLIZZARD_API_REGION = os.getenv("BLIZZARD_API_REGION").lower()

blizzard_token_url = f'https://{BLIZZARD_API_REGION}.battle.net/oauth/token'
blizzard_api_base_url = f'https://{BLIZZARD_API_REGION}.api.blizzard.com'

In [12]:
#  Authenticate on Blizzard OAuth & get a token

oauth_session = OAuth2Session(
    client_id=BLIZZARD_API_CLIENT_ID,
    client_secret=BLIZZARD_API_CLIENT_SECRET,
    token_endpoint=blizzard_token_url
)

try:
    oauth_token = oauth_session.fetch_token(
        blizzard_token_url,
        grant_type='client_credentials'
    )
    print(f"Successfully authenticated. Token expires in {oauth_token.get('expires_in')} seconds")
except Exception as e:
    print(f"Authentication failed: {e}")
    raise


Successfully authenticated. Token expires in 86399 seconds
